In [ ]:
# !pip install pandas-profiling==2.7.1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# from pandas_profiling import ProfileReport
from sklearn import preprocessing
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/house_prices/train.csv')
# test = pd.read_csv('/content/drive/MyDrive/house_prices/test.csv')
# sample = pd.read_csv('/content/drive/MyDrive/house_prices/sample_submission.csv')

In [ ]:
# prof = ProfileReport(train)
# prof.to_file(output_file='output.html')

In [ ]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
def pre_processing(df):
  # lots_of_zeros = ['MasVnrArea', 'BsmtFinSF2', 'LowQualFinSF', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF' 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
  list_miss_to_mean = ['LotFrontage']
  remove_cols = []
  list_remove_cols_missing = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']
  list_remove_cols_zeros = ['PoolArea','MiscVal']
  list_remove_rows_garage = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond','GarageCars', 'GarageArea']
  list_remove_rows_bsmt = ['BsmtExposure', 'BsmtQual', 'BsmtCond', 'BsmtFinType1','BsmtFinType2']
  list_remove_rows_little = ['MasVnrType', 'MasVnrArea', 'Electrical']

  if 'Alley' in df.columns:
    remove_cols = list_remove_cols_missing + list_remove_cols_zeros
    df = df.drop(remove_cols, axis=1)

  # df = df.dropna(subset=list_remove_rows_garage).dropna(subset=list_remove_rows_bsmt).dropna(subset=list_remove_rows_little)

  df[list_miss_to_mean[0]] = df.fillna(df[list_miss_to_mean[0]].mean())[list_miss_to_mean[0]]

  return df

In [ ]:
df = pre_processing(train)
# test = (test.set_index('Id').join(sample.set_index('Id')).reset_index())
# test = pre_processing(test)

In [ ]:
train, test = train_test_split(df, test_size=0.2)

In [ ]:
for name in list(train.columns):
  if train[name].isna().any():
    print("train | " + name + ' : ' + str(train[name].isna().any()))
  if test[name].isna().any():
    print("test | " + name + ' : ' + str(test[name].isna().any()))

train | MasVnrType : True
train | MasVnrArea : True
train | BsmtQual : True
test | BsmtQual : True
train | BsmtCond : True
test | BsmtCond : True
train | BsmtExposure : True
test | BsmtExposure : True
train | BsmtFinType1 : True
test | BsmtFinType1 : True
train | BsmtFinType2 : True
test | BsmtFinType2 : True
train | Electrical : True
train | GarageType : True
test | GarageType : True
train | GarageYrBlt : True
test | GarageYrBlt : True
train | GarageFinish : True
test | GarageFinish : True
train | GarageQual : True
test | GarageQual : True
train | GarageCond : True
test | GarageCond : True


In [ ]:
df = pd.concat([train, test[train.columns]]).dropna()
for name in list(df.columns):
  if df[name].isna().any():
    print("train | " + name + ' : ' + str(df[name].isna().any()))

In [ ]:
for name in list(df.dtypes[df.dtypes == object].index):
  # print(name +': '+ train[name][0])
  le = preprocessing.LabelEncoder()
  le.fit(df[name])
  df[name] = le.transform(df[name])

In [ ]:
train_final = df[df['Id'].isin(list(train['Id'].values))]
test_final = df[df['Id'].isin(list(test['Id'].values))]

In [ ]:
X_train = train_final.drop(['SalePrice','Id'], axis=1)#.apply(le.fit_transform)
y_train = train_final['SalePrice']

# .drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'PoolArea','MiscVal'], axis=1))

X_test = test_final.drop(['SalePrice','Id'], axis=1)#.apply(le.fit_transform)
X_test = test_final[X_train.columns]
y_test = test_final['SalePrice']
# X_test = X_test.dropna(subset=['MSZoning', 'Utilities', 'KitchenQual', 'Functional', 'SaleType'])

In [ ]:
# for name in list(X_train.dtypes[X_train.dtypes == object].index):
#   # print(name +': '+ train[name][0])
#   le.fit(X_train[name])
#   X_train[name] = le.transform(X_train[name])
#   X_test[name] = le.transform(X_test[name])
#   # le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
#   # print(le_name_mapping)
#   print(name)

In [ ]:
model = RandomForestRegressor(max_depth=10, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, random_state=42)

In [ ]:
model.score(X_train, y_train)

0.9750383881025538

In [ ]:
model.score(X_test, y_test)

0.8661440033038807